In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 15:25:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import date_format
from pyspark.sql.functions import to_date

23/12/04 15:25:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
import datetime

data = [
    (121, 'US', 'approved', 1000, datetime.date(2018, 12, 18)),
    (122, 'US', 'declined', 2000, datetime.date(2018, 12, 19)),
    (123, 'US', 'approved', 2000, datetime.date(2019, 1, 1)),
    (124, 'DE', 'approved', 2000, datetime.date(2019, 1, 7))
]

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("state", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("trans_date", DateType(), True)
])

df = spark.createDataFrame(data, schema)

df.show()
print(df.dtypes)

+---+-------+--------+------+----------+
| id|country|   state|amount|trans_date|
+---+-------+--------+------+----------+
|121|     US|approved|  1000|2018-12-18|
|122|     US|declined|  2000|2018-12-19|
|123|     US|approved|  2000|2019-01-01|
|124|     DE|approved|  2000|2019-01-07|
+---+-------+--------+------+----------+

[('id', 'int'), ('country', 'string'), ('state', 'string'), ('amount', 'int'), ('trans_date', 'date')]


In [10]:
df.createOrReplaceTempView("Transactions")

In [18]:
result = spark.sql(
    """
    select DATE_FORMAT(trans_date, 'y M') as month, country,
      count(id) as trans_count, 
      sum(case when state = 'approved' then 1 else 0 end) as approved_count,
      sum(amount) as trans_total_amount,
      sum(case when state = 'approved' then amount else 0 end) as approved_total_amount
    from Transactions
    group by 1, 2
    """
)
result.show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018 12|     US|          2|             1|              3000|                 1000|
| 2019 1|     US|          1|             1|              2000|                 2000|
| 2019 1|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+

